In [1]:
import requests
import zipfile
import re
import random

In [2]:
# data download, unzip
r = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
  f.write(r.content)
zipfile.ZipFile("sms.zip").extractall("./")

In [3]:
# load data
data = []
with open("SMSSpamCollection.txt", "r") as f:
  for line in f:
    cls, txt = line.strip().split("\t")
    bow = set(re.findall("[0-9a-z_]+", txt.lower()))
    data.append([cls, bow])

In [4]:
#split data into train and test
random.shuffle(data)
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train = data[:train_size]
test = data[train_size:]

In [5]:
# train 데이터에서, naive bayes 계산을 위한 값들을 미리 다 계산해놓기
# prior prob. P(spam), P(ham)
# P(spam) = spam 문자 수 / 전체 문자 수
# 라플라스 스무딩 -> 관측한 메일에서 한번도 등장하지 않은 단어가 있으면, 예측할 때 결과가 좀 이상해지는 문제!
# 가상의 spam문자, ham문자를 하나씩 추가. 추가한 spam, ham 문자에는 세상 존재하는 모든 단어가 다있음
# 가상의 spam문자, ham문자의 영향력이 지나치면 곤란하기 때문에, 가중치를 줄 수 있음 alpha
n_total = train_size
n_spam = sum(1 for cls, bow in train if cls == 'spam')
n_ham = sum(1 for cls, bow in train if cls == 'ham')

alpha = 0.00001

prior_spam = (n_spam + alpha) / (n_total + 2*alpha)
prior_ham = (n_ham + alpha) / (n_total + 2*alpha)

print(prior_spam, prior_ham)
# 모든 단어의 likelihood, P(w|spam), P(w|ham)

0.13254093010746384 0.8674590698925361


In [6]:
# 모든 단어의 likelihood, P(w|spam), P(w|ham)
# P(w|spam) = (w를 포함하는 spam 문자 수 + alpha) / (전체 spam 문자 수 + alpha)
# P(w|ham) = (w를 포함하는 ham 문자 수 + alpha) / (전체 ham 문자 수 + alpha)
from collections import Counter

spam_words = Counter(word for cls, bow in train
 for word in bow
 if cls == 'spam')

ham_words = Counter(word for cls, bow in train
 for word in bow
 if cls == 'ham')

In [7]:
def predict(bow):
  #  spam_score = P(spam) * P(w1 | spam) * P(w2 | spam) ...
  #  ham_score = P(ham) * P(w1 | ham) * P(w2 | ham) ...
  #  P(w|spam) = w포함한 spam 수 + alpha / 전체 spam 수 + alpha
  spam_score = prior_spam
  ham_score = prior_ham
  for word in bow:
    spam_score *= (spam_words[word] + alpha) / (n_spam + alpha)
    ham_score *= (ham_words[word] + alpha) / (n_ham + alpha)

  if spam_score < ham_score:
    return "ham"
  else:
    return "spam"

In [8]:
tp, tn, fp, fn = 0,0,0,0

for ans, bow in test:
  pred = predict(bow)
  if pred == 'spam':
    if ans == 'spam':
      tp += 1
    else: # ans == 'ham'
      fp += 1
  else: #pred == 'ham'
    if ans == 'ham':
      tn += 1
    else: # ans == 'spam'
      fn += 1

print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")

tp: 150, tn: 928, fp: 31, fn: 6


In [9]:
# accuracy = 맞힌 수 / 전체 테스트 수
acc = (tp + tn) / (tp +tn + fp + fn)
# precision = 스팸이야! 중 실제 스팸 수 / 스팸이야!
prec = tp / (tp + fp)
# recall = 내가 찾아낸 스팸 수 / 실제 스팸 수
recall = tp / (tp + fn)
# f1 = precision과 recall의 조화평균
f1 = 2 * prec * recall / (prec + recall)
print(f"acc: {acc:.3f}, prec: {prec:.3f}, recall: {recall:.3f}, f1: {f1:.3f}") 

acc: 0.967, prec: 0.829, recall: 0.962, f1: 0.890
